In [1]:
import pandas as pd

In [2]:
import numpy as np
import re

In [3]:
import nltk

In [4]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.tag import pos_tag
from nltk import ne_chunk
from nltk.chunk import conlltags2tree, tree2conlltags
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim.models import Word2Vec
from textblob import TextBlob
import emoji

In [5]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/kaushalpancholi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kaushalpancholi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/kaushalpancholi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/kaushalpancholi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/kaushalpancholi/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/kaushalpancholi/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [21]:
file_path = "/home/kaushalpancholi/Documents/6000+ Indian Food Recipes Dataset/IndianFoodDatasetCSV.csv"
df = pd.read_csv(file_path)


In [22]:
print(df)

       Srno                                         RecipeName  \
0         1                               Masala Karela Recipe   
1         2  टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Re...   
2         3  Ragi Semiya Upma Recipe - Ragi Millet Vermicel...   
3         4  Gongura Chicken Curry Recipe - Andhra Style Go...   
4         5  आंध्रा स्टाइल आलम पचड़ी रेसिपी - Adrak Chutney ...   
...     ...                                                ...   
6866  14073  गोअन मशरुम जकुटी रेसिपी - Goan Mushroom Xacuti...   
6867  14107  शकरकंदी और मेथी का पराठा रेसिपी - Sweet Potato...   
6868  14165      Ullikadala Pulusu Recipe | Spring Onion Curry   
6869  14167  Kashmiri Style Kokur Yakhni Recipe-Chicken Coo...   
6870  14211              नवरंग दाल रेसिपी - Navrang Dal Recipe   

                                   TranslatedRecipeName  \
0                                  Masala Karela Recipe   
1                            Spicy Tomato Rice (Recipe)   
2     Ragi Semiya Upma Recipe 

In [23]:
print(list(df.columns))

['Srno', 'RecipeName', 'TranslatedRecipeName', 'Ingredients', 'TranslatedIngredients', 'PrepTimeInMins', 'CookTimeInMins', 'TotalTimeInMins', 'Servings', 'Cuisine', 'Course', 'Diet', 'Instructions', 'TranslatedInstructions', 'URL']


In [24]:
print(df.head())

   Srno                                         RecipeName  \
0     1                               Masala Karela Recipe   
1     2  टमाटर पुलियोगरे रेसिपी - Spicy Tomato Rice (Re...   
2     3  Ragi Semiya Upma Recipe - Ragi Millet Vermicel...   
3     4  Gongura Chicken Curry Recipe - Andhra Style Go...   
4     5  आंध्रा स्टाइल आलम पचड़ी रेसिपी - Adrak Chutney ...   

                                TranslatedRecipeName  \
0                               Masala Karela Recipe   
1                         Spicy Tomato Rice (Recipe)   
2  Ragi Semiya Upma Recipe - Ragi Millet Vermicel...   
3  Gongura Chicken Curry Recipe - Andhra Style Go...   
4  Andhra Style Alam Pachadi Recipe - Adrak Chutn...   

                                         Ingredients  \
0  6 Karela (Bitter Gourd/ Pavakkai) - deseeded,S...   
1  2-1/2 कप चावल - पका ले,3 टमाटर,3 छोटा चमच्च बी...   
2  1-1/2 cups Rice Vermicelli Noodles (Thin),1 On...   
3  500 grams Chicken,2 Onion - chopped,1 Tomato -...   
4  1 बड़ा च

In [12]:
# def preprocess_text(text):
#     # Tokenization
#     tokens = word_tokenize(text)
    
#     # Convert to lowercase
#     tokens = [token.lower() for token in tokens]
    
#     # Remove punctuation
#     tokens = [token for token in tokens if token.isalpha()]
    
#     # Remove stopwords
#     stop_words = set(stopwords.words('english'))
#     tokens = [token for token in tokens if token not in stop_words]
    
#     # Lemmatization
#     lemmatizer = WordNetLemmatizer()
#     tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
#     return ' '.join(tokens)


In [13]:
# df['processed_recipe'] = df['TranslatedRecipeName'].apply(preprocess_text)
# print(df[['TranslatedRecipeName', 'processed_recipe']].head())

In [30]:
# Function to clean text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    # text = emoji.get_emoji_regexp().sub(u'', text)  # Remove emojis
    return text

In [31]:
# Function to tokenize text
def tokenize(text):
    return word_tokenize(text)

In [32]:
# Function to remove stopwords
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    return [token for token in tokens if not token in stop_words]

In [33]:
# Function to perform lemmatization
def lemmatize(tokens):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in tokens]

In [34]:
# Function to perform POS tagging
def pos_tagging(tokens):
    return pos_tag(tokens)

In [35]:
# Function for named entity recognition
def named_entity_recognition(pos_tags):
    tree = conlltags2tree(tree2conlltags(ne_chunk(pos_tags)))
    return tree

In [36]:
# Select the columns to preprocess
columns_to_preprocess = ['Ingredients', 'TranslatedIngredients']

In [37]:
# Apply the preprocessing steps to the columns
for column in columns_to_preprocess:
    df[column + '_clean'] = df[column].astype(str).apply(clean_text)
    df[column + '_tokens'] = df[column + '_clean'].apply(tokenize)
    df[column + '_no_stopwords'] = df[column + '_tokens'].apply(remove_stopwords)
    df[column + '_lemmatized'] = df[column + '_no_stopwords'].apply(lemmatize)
    df[column + '_pos_tags'] = df[column + '_lemmatized'].apply(pos_tagging)
    df[column + '_ner'] = df[column + '_pos_tags'].apply(named_entity_recognition)

In [38]:
# Example of vectorization with Bag of Words
count_vectorizer = CountVectorizer()
df['Ingredients_bow'] = list(count_vectorizer.fit_transform(df['Ingredients_lemmatized'].apply(lambda x: ' '.join(x))))

In [39]:
# Example of using Word2Vec for embeddings
word2vec_model = Word2Vec(df['Ingredients_lemmatized'], min_count=1)